In [2]:
import cv2
import tensorflow as tf
import pandas as pd
import math
import numpy as np
import os

from tqdm import tqdm

from random import shuffle
# from matplotlib import pyplot as plt

from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import Sequence, to_categorical

from python_speech_features import mfcc, logfbank
from scipy.io import wavfile

In [3]:
AUDIOS_DIR = os.path.join("../datasets", "voxceleb1_wavfile")
num_classes = 300

def train_test_split():
    labels = sorted(os.listdir(AUDIOS_DIR))[:num_classes]
    train_files = []
    eval_files = []
    for label in labels:
        folder = os.path.join(AUDIOS_DIR, label)
        files = [os.path.join(folder, file) for file in os.listdir(folder) if file.endswith(".wav")]
        shuffle(files)
        split_idx = int(len(files) * 0.7)
        train_files.extend(files[:split_idx])
        eval_files.extend(files[split_idx:])
    return train_files, eval_files, labels

train_files, eval_files, labels = train_test_split()
print("Train files: {}".format(len(train_files)))
print("Eval files: {}".format(len(eval_files)))
print("Labels: {}".format(len(labels)))

Train files: 900
Eval files: 600
Labels: 300


In [4]:
def plot_signals(signals):
    nrows = len(signals)
    ncols = 1
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=False, sharey=True, figsize=(10, 20))
    fig.suptitle("Time Series", size=16)
    for x in range(nrows):
        axes[x].set_title(list(signals.keys())[x])
        axes[x].plot(list(signals.values())[x])
        axes[x].get_xaxis().set_visible(False)
        axes[x].get_yaxis().set_visible(True)
    fig.set_tight_layout(True)
    plt.show()

In [5]:
signals = {}
folder = os.listdir(AUDIOS_DIR)[156]
folder = os.path.join(AUDIOS_DIR, folder)
for file in os.listdir(folder):
    filepath = os.path.join(folder, file)
    if file.endswith(".wav"):
        signals[filepath] = wavfile.read(filepath)[1]
# plot_signals(signals)

NameError: name 'plt' is not defined

In [6]:
numcep = 13
nfilt = 26
nfft = 512
threshold = 200
# n_channels = 1
n_channels = 3

def features_from_audio(audio_file, secs=0.1, n_samples=5):
    def envelop(signal, rate, threshold):
        mask = []
        y = pd.Series(signal).apply(np.abs)
        y_mean = y.rolling(window=int(rate/10), min_periods=1, center=True).mean()
        for mean in y_mean:
            if mean > threshold:
                mask.append(True)
            else:
                mask.append(False)
        return mask
    rate, signal = wavfile.read(audio_file)
    mask = envelop(signal, rate, threshold)
    signal = signal[mask]
    step = int(rate*secs)
    samples = []
    for i in range(n_samples):
        rand_idx = np.random.randint(0, signal.shape[0]-step)
        sample = signal[rand_idx:rand_idx+step]
        sample = mfcc(sample, rate, numcep=numcep, nfilt=nfilt, nfft=nfft).astype(np.float32)
        sample = np.expand_dims(sample, axis=-1)
        if n_channels == 3:
            sample = np.tile(sample, [1, 1, 3])
        samples.append(sample)
    return samples

In [8]:
train_audio_features = []
train_labels = []

eval_audio_features = []
eval_labels = []

n_samples_per_file = 5
# files = train_files + eval_files
# files = ['../datasets\\voxceleb1_wavfile\\157\\00004.wav']
failures = []
for file in tqdm(train_files):
    try:
        samples = features_from_audio(file, 0.1, n_samples_per_file)
    except Exception as e:
        failures.append(file)
        print(e)
    else:
        train_audio_features.extend(samples)
        train_labels.extend([os.path.basename(os.path.dirname(file)) for _ in range(n_samples_per_file)])
        
for file in tqdm(eval_files):
    try:
        samples = features_from_audio(file, 0.1, n_samples_per_file)
    except Exception as e:
        failures.append(file)
        print(e)
    else:
        eval_audio_features.extend(samples)
        eval_labels.extend([os.path.basename(os.path.dirname(file)) for _ in range(n_samples_per_file)])

print("Failures:", failures)
print(len(train_audio_features), len(train_labels))
print(len(eval_audio_features), len(eval_labels))

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [01:08<00:00,  8.72it/s]

Failures: []
4500 4500
3000 3000


In [9]:
len(eval_audio_features)

3000

In [14]:
class PersonIDSequence(Sequence):

    def __init__(self, features, labels, unique_labels, batch_size):
        self.features = features
        self.labels = labels
        self.unique_labels = unique_labels
#         shuffle(self.features)
        self.num_labels = len(self.unique_labels)
        self.batch_size = batch_size
        print(len(self.features), math.ceil(len(self.features) / self.batch_size))

    def __len__(self):
        return math.ceil(len(self.features) / self.batch_size)

    def __getitem__(self, idx):
        features = np.array(self.features[idx * self.batch_size:(idx + 1) * self.batch_size])
        y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        y_indices = [to_categorical(self.unique_labels.index(i), num_classes=self.num_labels)
                     for i in y]
        return features, np.array(y_indices)

    def on_epoch_end(self):
#         shuffle(self.files)
#         shuffler = np.random.permutation(len(self.features))
#         self.features = self.features[shuffler]
#         self.labels = self.labels[shuffler]
        pass


In [21]:
# def audio_model():
#     input_ = layers.Input(shape=(9, 13, 1))
#     x = layers.Conv2D(256, 3, strides=(1, 1), padding='same')(input_)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
    
#     x = layers.Conv2D(256, 3, strides=(1, 1), padding='same')(input_)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
    
#     x = layers.Conv2D(256, 3, strides=(1, 1), padding='same')(input_)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
    
#     x = layers.MaxPooling2D((2, 2))(x)
#     x = layers.Flatten()(x)
    
# #     x = layers.Dropout(0.5)(x)
# #     x = layers.Dense(1024, activation='relu')(x)
    
#     x = layers.Dropout(0.5)(x)
#     x = layers.Dense(300, name='last_dense')(x)
#     x = layers.Softmax()(x)

#     model = Model(inputs=input_, outputs=x, name='audio_model_cnn')
#     return model

# model = audio_model()
# # model.summary()

In [ ]:
def audio_model():
    vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    for layer in vgg16.layers:
        layer.trainable = False
    x = vgg16.output
    x = layers.Flatten()(x)                                # Flatten dimensions to for use in FC layers
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dropout(0.2)(x)                             # Dropout layer to reduce overfitting
    x = layers.Dense(len(labels), name="last_dense")(x) 
    x = layers.Softmax()(x)                                # Softmax for multiclass
    return Model(inputs=vgg16.input, outputs=x)

model = audio_model()
# model.summary()

In [16]:
unique_labels = sorted(np.unique(train_labels))
train_ds = PersonIDSequence(train_audio_features, train_labels, unique_labels, batch_size=64)
eval_ds = PersonIDSequence(eval_audio_features, eval_labels, unique_labels, batch_size=64)

4500 71
3000 47


In [18]:
epochs = 80
lr = 0.001

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(lr))
history = model.fit(train_ds, epochs=epochs, validation_data=eval_ds)

71/71 [==============================] - 8s 107ms/step - loss: 4.2470 - accuracy: 0.1669 - val_loss: 6.0239 - val_accuracy: 0.0797


In [12]:
model.evaluate(eval_ds)

47/47 [==============================] - 0s 4ms/step - loss: 9.5335 - accuracy: 0.0893


[9.533499859749003, 0.08933333]

In [20]:
model.save("audio_model_e{}_lr{}.h5".format(epochs, lr))

In [14]:
model.summary()

Model: "audio_model_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 9, 13, 1)]        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 13, 256)        2560      
_________________________________________________________________
batch_normalization_2 (Batch (None, 9, 13, 256)        1024      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 9, 13, 256)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 6, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6144)              0         
_________________________________________________________________
dropout (Dropout)            (None, 6144)          